In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib_venn import venn2, venn2_circles
import argparse
from create_reference_from_tsv_and_pepxml import ReferenceWriter
from sensitivity_calculator import SensitivityAndSpecificity
from create_PSM_df import PSM_FDR
from Bio import SeqIO
from collections import defaultdict
from Bio import pairwise2 as pw2
import wget
from ReadAccTaxon import ReadAccTaxon
# load taxon graph
import sys  
sys.path.insert(0, '/home/jules/tax2proteome_projects/tax2proteome/')
from TaxonGraph import TaxonGraph
taxon_graph = TaxonGraph()
taxon_graph.create_graph("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/taxdump.tar.gz")

In [ ]:
# Files
#new_reduced: ohne +1 charge specta
uniprot_nr_species_tsv_with_wrong_error = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_new_reduced.tsv"


reference_tsv_with_kleiner_db = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_db/Run1_U1_2000ng.t.xml_reduced.tsv"
reference_tsv_with_aradiopsis = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_reference_aradiopsis/Run1_U1_2000ng_kleiner_aradiopsis.t.xml_reduced.tsv"
reference_tsv_with_ara_yeast_human = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_ara_yeast_human/Run1_U1_2000ng.t.xml_new_reduced.tsv"
path_to_bachelor_results = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/"
path_to_uniprot_reanalysis = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_reanalysis_uniprot"
path_to_tanca_results_bachelor = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv"
path_to_kleiner_species_nr_0_9_identity = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/uniprot_species_sequences_0_9_identity/Run1_U1_2000ng_uniprot_species_0_9_identity.t.xml.tsv"
path_to_kleiner_species_nr_0_9_identity_reduced = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/uniprot_species_sequences_0_9_identity/Run1_U1_2000ng_uniprot_species_0_9_identity.t.xml_new_reduced.tsv"
def get_path_to_refernce_analysis_results(fdr):
    return f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_db/Run1_U1_2000ng.t.xml_reduced.tsv_{fdr}_sensitivity_new"

def get_path_to_refernce_analysis_results_with_aradiopsis(fdr):
    return f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_reference_aradiopsis/Run1_U1_2000ng_kleiner_aradiopsis.t.xml_reduced.tsv_{fdr}_sensitivity_new"



uniprot_reanalysis = {'species': path_to_uniprot_reanalysis + "/species/Run1_U1_2000ng_uniprot_species.t.xml_new_reduced.tsv",
                      'species_nr': path_to_uniprot_reanalysis + "/species_nr/Run1_U1_2000ng_uniprot_species_nr.t.xml_reduced.tsv",
                      "species_species": path_to_uniprot_reanalysis + "/species_species/Run1_U1_2000ng_uniprot_species_species.t.xml_reduced.tsv",
                      'genus_nr': path_to_uniprot_reanalysis + "/genus_nr/Run1_U1_2000ng_uniprot_genus_nr.t.xml_reduced.tsv"
                     }
    
uniprot_nr_reduced_tsv = {
    'subspecies': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus_nr.t.xml_new_reduced.tsv",
    'family': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_family_nr.t.xml_new_reduced.tsv",
}
    
uniprot_reduced_tsv = {
    'subspecies': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus.t.xml_new_reduced.tsv"
}

uniprot_species_reduced_tsv =  {
    'subspecies': path_to_uniprot_reanalysis + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_species.t.xml_new_reduced.tsv",
    'genus':  path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus_species.t.xml_new_reduced.tsv"
}

ncbi_reduced_tsv= {
    'subspecies': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_genus.t.xml_new_reduced.tsv"
}

ncbi_species_reduced_tsv = {
    'species': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_genus_species.t.xml_new_reduced.tsv"
}
swissprot_reduced_tsv = {
    'species': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_genus.t.xml_new_reduced.tsv",
    'family': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_family.t.xml_new_reduced.tsv",
    'order': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_order.t.xml_new_reduced.tsv"
}

tanca_ncbi_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_Tanca_species.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_tanca_family.t.xml_new_reduced.tsv",
}

tanca_uniprot_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_species_nr.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_family_nr.t.xml_new_reduced.tsv",
    'order': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_order_nr.t.xml_new_reduced.tsv"
}

tanca_swissprot_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_species.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_family.t.xml_new_reduced.tsv",
    'order': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_order.t.xml_new_reduced.tsv"
}

def get_dict_databases_to_size_and_result_file(fdr):
    dict_databases_to_size_and_result_file={'reference': (123088, f"{reference_tsv_with_kleiner_db}_{fdr}_sensitivity_new"),
                                        "reference_with_aradiopsis": (138980, f"{reference_tsv_with_aradiopsis}_{fdr}_sensitivity_new"),
                                        
                                        'ncbi_subspecies': (300000, f"{ncbi_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        'ncbi_species': (8702135, f"{ncbi_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "ncbi_genus":(27804893, f"{ncbi_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        
                                        "uniprot_subspecies": (4683371, f"{uniprot_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        "uniprot_species": (4683371, f"{uniprot_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_species_species": (2093157, f"{uniprot_species_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_subspecies_nr": (200000, f"{uniprot_nr_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        "uniprot_species_nr": (2991727, f"{uniprot_nr_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus": (18352148,  f"{uniprot_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus_species": (13068285,  f"{uniprot_species_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus_nr": (13210287,  f"{uniprot_nr_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_family_nr": (22509624, f"{uniprot_nr_reduced_tsv['family']}_{fdr}_sensitivity_new"),
                                        
                                        "swissprot_species": (58505, f"{swissprot_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "swissprot_genus": (88164, f"{swissprot_reduced_tsv['genus']}_{fdr}_sensitivity_new" ),
                                        "swissprot_family": (124044, f"{swissprot_reduced_tsv['family']}_{fdr}_sensitivity_new"),
                                        "swissprot_order": (181725, f"{swissprot_reduced_tsv['order']}_{fdr}_sensitivity_new")
                                       }
    return dict_databases_to_size_and_result_file

dict_databases_to_size_and_result_file = get_dict_databases_to_size_and_result_file(0.05)
uniprot_dict =dict(filter(lambda item: item[0] in ['uniprot_subspecies', 'uniprot_species', 'uniprot_genus'], dict_databases_to_size_and_result_file.items()))
uniprot_nr_dict =dict(filter(lambda item: '_nr' in item[0], dict_databases_to_size_and_result_file.items()))
uniprot_species_dict=dict(filter(lambda item: item[0] in ['uniprot_species_species', 'uniprot_genus_species'], dict_databases_to_size_and_result_file.items()))
ncbi_dict =dict(filter(lambda item: 'ncbi' in item[0], dict_databases_to_size_and_result_file.items()))
swissprot_dict=dict(filter(lambda item: 'swiss' in item[0], dict_databases_to_size_and_result_file.items()))

In [ ]:
example_df = pd.DataFrame({'Title': ["Run1_U1_2000ng.150015.150015.2", "Run1_U1_2000ng.125600.125600.2", "Run1_U1_2000ng.74833.74833.2", "Run1_U1_2000ng.149614.149614.2", "Run1_U1_2000ng.112688.112688.2", "Run_2TIME"],\
                           'Peptide':["AALESTLAAITESLKEGDAVQLVGFGTFK", "APEELAGKTEEDALVAYLQGLGLNR", "IEIGKENTIIIDGAGDASAIEGR", "AALESTLAAITESLKEGDAVQLVGFGTFK", "ALGLYQQFDEDKGVQDVGAALATLR", "DEFNITVE2TIMES"], \
                           "Hyperscore":[126.4, 126.3, 126.2, 126.1, 126.0, 100], \
                           "Protein": [{'tr|A0A1E8CPD5|A0A1E8CPD5_SALET', 'tr|A0A484Y9D5|A0A484Y9D5_ECOLX', 'tr|A0A4U9LQT0|A0A4U9LQT0_ECOLX', 'tr|A0A376YHH4|A0A376YHH4_ECOLX', 'tr|W1W199|W1W199_ECOLX', 'tr|A0A3V3ZGB0|A0A3V3ZGB0_SALET', 'tr|A0A4Y5XXS5|A0A4Y5XXS5_SALET', 'tr|A0A3Y1X2E6|A0A3Y1X2E6_SALHO', 'tr|G5QBB1|G5QBB1_SALMO', 'tr|A0A4T8E0J6|A0A4T8E0J6_ECOLX', 'tr|A0A376I4V6|A0A376I4V6_ECOLX', 'tr|W1HKQ7|W1HKQ7_ECOLX', 'tr|A0A370YFP7|A0A370YFP7_ECOLX', 'tr|A0A3X0JPQ0|A0A3X0JPQ0_SALER', 'tr|A0A4T3RZ71|A0A4T3RZ71_ECOLX', 'tr|A0A4S4ZSU8|A0A4S4ZSU8_ECOLI', 'tr|A0A379SBZ2|A0A379SBZ2_SALER', 'tr|A0A3V9Z5B5|A0A3V9Z5B5_SALET', 'tr|A0A0T9Y4H9|A0A0T9Y4H9_SALET', 'tr|A0A210G7C8|A0A210G7C8_ECOLX', 'tr|G5S3V5|G5S3V5_SALET'}, {'tr|A0A132HPF0|A0A132HPF0_9BURK'}, {'tr|A0A132HK13|A0A132HK13_9BURK', 'tr|A0A2L0WY25|A0A2L0WY25_9BURK'}, {'tr|A0A1E8CPD5|A0A1E8CPD5_SALET', 'tr|A0A484Y9D5|A0A484Y9D5_ECOLX', 'tr|A0A4U9LQT0|A0A4U9LQT0_ECOLX', 'tr|A0A376YHH4|A0A376YHH4_ECOLX', 'tr|W1W199|W1W199_ECOLX', 'tr|A0A3V3ZGB0|A0A3V3ZGB0_SALET', 'tr|A0A4Y5XXS5|A0A4Y5XXS5_SALET', 'tr|A0A3Y1X2E6|A0A3Y1X2E6_SALHO', 'tr|G5QBB1|G5QBB1_SALMO', 'tr|A0A4T8E0J6|A0A4T8E0J6_ECOLX', 'tr|A0A376I4V6|A0A376I4V6_ECOLX', 'tr|W1HKQ7|W1HKQ7_ECOLX', 'tr|A0A370YFP7|A0A370YFP7_ECOLX', 'tr|A0A3X0JPQ0|A0A3X0JPQ0_SALER', 'tr|A0A4T3RZ71|A0A4T3RZ71_ECOLX', 'tr|A0A4S4ZSU8|A0A4S4ZSU8_ECOLI', 'tr|A0A379SBZ2|A0A379SBZ2_SALER', 'tr|A0A3V9Z5B5|A0A3V9Z5B5_SALET', 'tr|A0A0T9Y4H9|A0A0T9Y4H9_SALET', 'tr|A0A210G7C8|A0A210G7C8_ECOLX', 'tr|G5S3V5|G5S3V5_SALET'}, {'tr|A0A2L0XCX6|A0A2L0XCX6_9BURK', 'tr|Q1LI33|Q1LI33_CUPMC', 'tr|A0A482IVN3|A0A482IVN3_9BURK'}, {'tr|A0A132HPF0|A0A132HPF0_9BURK'}],\
                           "decoy": [{False} , {False} , {False} , {False} , {False} , {False} ], \
                           'taxID_species': [{562, 28901}, {119219}, {119219}, {562, 28901}, {119219}, {119219}], \
                           })

example_df

In [ ]:
def get_taxa_rows(column, taxID):
    if type(taxID)==int:
        return [True if taxID in t_set else False for t_set in column]
    elif type(taxID)==list:
        return [True if len(set(taxID).intersection(set(t_set)))>0 else False for t_set in column]
    
def get_exclusive_rows(column, taxid):
    if type(taxid)==int:
        return [True if {taxid} == t_set else False for t_set in column]
    elif type(taxid)==list:
        return [True if len(set(t_set).difference(taxid))==0 else False for t_set in column]
    
def get_hit_rows2(decoy_column):
    return [True if d_set in  [{True, False}, {False}] else False for d_set in decoy_column]
    

In [ ]:
def get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra=False, columns=None):
    cs = ['Protein', 'Hyperscore', 'decoy', 'taxID']
    if columns:
        cs = cs + columns
    reduced_df = ReferenceWriter.read_csv_with_generic_function(file, cs, remove_one_charged_spectra)
    fdr_pos_result, number_psm_result, number_decoy_result, double_spectra_result, score_last_item_result = PSM_FDR.determine_FDR_position(reduced_df, fdr)
    return number_psm_result, reduced_df[0:fdr_pos_result]

def get_df_in_fdr_without_decoy(file, fdr, remove_one_charged_spectra=True, columns=None):
    df = get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra, columns)[1]
    df = df[get_hit_rows2(df.decoy)]
    return df

In [ ]:
def get_accs(tax):
    all_descending_taxa = taxon_graph.find_taxIDs(tax)
    #print(sorted(list(all_descending_taxa)))
    acc_set = set()
    with open("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/acc2tax_uniprot", 'r') as infile:
        for line in infile:
            fields = line.split('\t')
            try: 
                taxon = int(fields[1].strip())
                if taxon in all_descending_taxa:
                    acc_set.add(fields[0].strip()) 
            # empty lines
            except:
                continue
    return acc_set

def get_taxa_specific_accs(taxids):
    if type(taxids)==int:
        taxa_specific_accs = get_accs(taxids)
    elif type(taxids)==list:
        taxa_specific_accs = set()
        for taxon in taxids:
            taxa_specific_accs = taxa_specific_accs.union(get_accs(taxon))
    return taxa_specific_accs

In [ ]:
def taxon_exclusive_spectra(df, column_name, taxon):
    df_all_taxa_spec_identified_spectra = df[get_taxa_rows(df[column_name], taxon)]
    all_identified_spectra = set(df_all_taxa_spec_identified_spectra.Title)
    df_exclusive_spectra = df[get_exclusive_rows(df_all_taxa_spec_identified_spectra[column_name], taxon)]
    exclusive_spectra = set(df_exclusive_spectra.Title)
    df_all_taxa_spec_identified_spectra_no_one_hits = remove_all_accs_with_only_one_peptide(df_all_taxa_spec_identified_spectra)
    all_identified_spectra_no_one_hits = set(df_all_taxa_spec_identified_spectra_no_one_hits.Title)
    df_exclusive_spectra_no_one_hits = df[get_exclusive_rows(df_all_taxa_spec_identified_spectra_no_one_hits[column_name], taxon)]
    exclusive_spectra_no_one_hits = set(df_exclusive_spectra_no_one_hits.Title)
    return all_identified_spectra, all_identified_spectra_no_one_hits, exclusive_spectra, exclusive_spectra_no_one_hits

def taxon_exclusive_spectra_without_one_peptide(df, column_name, taxon):
    df_all_identified_spectra = df[get_taxa_rows(df[column_name], taxon)]
    df_all_identified_spectra = remove_all_accs_with_only_one_peptide(df)
    all_identified_spectra = set(df_all_identified_spectra.Title)
    df_exclusive_spectra = df[get_exclusive_rows(df[column_name], taxon)]
    exclusive_spectra = set(df_exclusive_spectra.Title)
    return all_identified_spectra, exclusive_spectra

def spectra_identified_in_df(df_species, df_family, taxon_species, taxon_family):
    species_spectra = set(df_species[get_taxa_rows(df_species.taxID_species, taxon_species)].Title)
    family_spectra = set(df_family[get_taxa_rows(df_family.taxID_family, taxon_family)].Title)
    new_identified_spectra = family_spectra.difference(species_spectra)
    all_species_spectra = set(df_species.Title)
    spectra_in_species_df = new_identified_spectra.intersection(all_species_spectra)
    return new_identified_spectra, spectra_in_species_df

def get_acc_to_taxid_dict(all_accs):
    acc_to_taxid_dict = {}
    with open("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/acc2tax_uniprot", 'r') as infile:
        for line in infile:
            fields = line.split('\t')
            try: 
                taxid = int(fields[1].strip())
                acc = fields[0].strip()
                if acc in all_accs:
                    acc_to_taxid_dict[acc]=taxid 
            # empty lines
            except:
                continue
    return acc_to_taxid_dict
    
def get_taxids_of_accs(acc_list, level, taxon_graph, acc_to_taxid_dict):
    taxid_list = []
    for acc in acc_list:
        taxid_list.append(get_taxid_of_acc(acc, level, taxon_graph, acc_to_taxid_dict))
    return taxid_list

def get_taxid_of_acc(acc, level, taxon_graph, acc_to_taxid_dict):
    try:
        taxid = taxon_graph.find_level_up(acc_to_taxid_dict[acc], level)
    except KeyError:
        taxid = "CRAP"
    return taxid
                                              
def sort_taxid_and_acc_in_df(df, level, acc_to_taxid_dict):
    df.Protein = df.Protein.apply(lambda acc_set: sorted(list(acc_set)))
    df[f"taxID_{level}"] = df.Protein.apply(lambda acc_list: get_taxids_of_accs(acc_list, level, taxon_graph, acc_to_taxid_dict))
    return df


In [ ]:
def test_sort_taxid_and_acc_in_df(level):
    acc_to_taxid_dict = {'a1':1, 'a2':2, 'a3':3, 'a4': 4}
    taxid_to_fam_taxid = {1: 11, 2:22, 3:33, 4:44}
    df = pd.DataFrame({'taxID_family': [{1,2,4}, {2,3}, {4,1}], 'Protein': [{"a3", "a2", "a1"}, {"a2", "a3"}, {"a1", "a4"}]})
    df.Protein = df.Protein.apply(lambda acc_set: sorted(list(acc_set)))
    print(df)
    df[f"taxID_{level}"] = df.Protein.apply(lambda acc_list: [acc_to_taxid_dict[acc] for acc in acc_list])
    print(df)
test_sort_taxid_and_acc_in_df('family')

In [ ]:
def remove_acc_row(column, acc_to_remove_set):
    return [False if len(set(accs).difference(acc_to_remove_set))==0 else True for accs in column]            

def remove_empty_rows(column):
    return [False if len(accs)==0 else True for accs in column]


def remove_all_once_appearing_accs(df, level):
    #print(len(df))
    all_accs = [item for sublist in df.Protein for item in sublist]
    all_accs.sort()
    #print("all_accs: ", len(set(all_accs)))
    acc_before=""
    acc_to_keep=set()
    for acc in all_accs:
        if acc == acc_before:
            acc_to_keep.add(acc)
        acc_before = acc
    print("acc_to_keep ", len(acc_to_keep))
 #   accs_to_remove = set(all_accs).difference(acc_to_keep)
  #  print("accs_to_remove: ", len(accs_to_remove))
    # find indices and remove taxid and protein acc
  #  for index, row in df.iterrows():
   #     print(row['Protein'], row[f'taxID_{level}'])   
    df.Protein = df.Protein.apply(lambda acc_list: set(acc_list).intersection(acc_to_keep))
    df[f"taxID_{level}"] = df[f"taxID_{level}"].apply(lambda tax_list: set(tax_list))
    df=df[remove_empty_rows(df.Protein)]
    return df


def remove_all_accs_with_only_one_peptide(df_taxon_spec):
    acc_to_peptide_dict=defaultdict(set)
    accs_to_remove= set()
    accs_to_keep = set()
    for index, row in df_taxon_spec.iterrows():
        for acc in row['Protein']:
            acc_to_peptide_dict[acc].add(row["Peptide"])
    for acc, pep_set in acc_to_peptide_dict.items():
        if len(pep_set)<2:
            accs_to_remove.add(acc)
        else:
            accs_to_keep.add(acc)
    df_taxon_spec=df_taxon_spec[remove_acc_row(df_taxon_spec.Protein, accs_to_remove)]
    df_taxon_spec.Protein = df_taxon_spec.Protein.apply(lambda acc_list: set(acc_list).intersection(accs_to_keep))
    df_taxon_spec = df_taxon_spec[remove_empty_rows(df_taxon_spec.Protein)]
    return df_taxon_spec

In [ ]:
virus_family_IDs = [10744, 10699, 10662, 11989]
virus_species_IDs = [10754, 101570, 1283336, 12022, 1977402]
df_virus_fam = df_in_fdr_uniprot_family_without_one_hits[get_taxa_rows(df_in_fdr_uniprot_family_without_one_hits.taxID_family, virus_family_IDs)]
df_exclusive_virus_fam = df_in_fdr_uniprot_family_without_one_hits[get_exclusive_rows(df_in_fdr_uniprot_family_without_one_hits.taxID_family, virus_family_IDs)]
df_without_one_hit_peptides = remove_all_accs_with_only_one_peptide(df_virus_fam)
df_without_one_hit_peptides

In [ ]:
def count_taxa(df_column):
    taxa_count_dict = {}
    for tax_set in df_column:
        for taxon in tax_set:
            if taxon in taxa_count_dict:
                taxa_count_dict[taxon]+= 1
            else:
                taxa_count_dict[taxon]=1
    return taxa_count_dict

def count_accs(acc_list):
    acc_list.sort()
    count_dict={1:0, 2:0, 3:0, '>3':0}
    try:
        acc_before = acc_list[0]
        count=0
        for acc in acc_list:
            if acc == acc_before:
                count+=1
            else:
                if count > 3:
                    count_dict['>3'] = count_dict['>3']+1
                else:
                    count_dict[count] = count_dict[count]+1
                count=1    
            acc_before = acc
        return count_dict
    except IndexError:
        return count_dict

def get_index_list(tax_list, taxids):
    return [i for i, x in enumerate(tax_list) if x in taxids]

def get_taxon_specific_df(df, level, taxids, acc_2_taxid_dict):
    df = df[get_taxa_rows(df[f"taxID_{level}"], taxids)]
    df.Protein = df.Protein.apply(lambda acc_set: [acc for acc in acc_set if \
                                                   get_taxid_of_acc(acc, level, taxon_graph, acc_2_taxid_dict) in taxids])

    df[f"taxID_{level}"] = df.Protein.apply(lambda acc_list: [get_taxid_of_acc(acc, level, taxon_graph, acc_2_taxid_dict) for acc in acc_list]) 
   #  df.Protein = df.Protein.apply(lambda acc_list:[x for i, x in enumerate(acc_list) if i in get_index_list(df[f"taxID_{level}"], taxids)])       
   # df[f"taxID_{level}"]= df[f"taxID_{level}"].apply(lambda tax_list:[taxid for taxid in tax_list if taxid in taxids])
    #remove rows with no accs
    df = df[remove_empty_rows(df.Protein)]
    if df.empty:
        df = pd.DataFrame(columns=["Title", "Protein", f"taxID_{level}"])
    return df
    
    
def describe_difference_between_species_and_family_identification(name, taxids_species, taxid_family,\
                                                                  df_in_fdr_species, df_in_fdr_family, \
                                                                  df_in_fdr_species_without_one_hits, \
                                                                  df_in_fdr_family_without_one_hits, acc_to_taxid_dict):
    print(name)
    for df_spec, df_fam, name in [(df_in_fdr_species, df_in_fdr_family, 'FULL'), (df_in_fdr_species_without_one_hits, df_in_fdr_family_without_one_hits, "REDUCED")]:
        if name=='FULL':
            print("RESULTS FOR FULL DF")
        else:
            print()
            print("RESULS FOR DF without ONE HITS")
        # keep only specific accs
        df_in_fdr_species_tax_acc = df_spec.copy(deep=True)
        df_in_fdr_family_tax_acc = df_fam.copy(deep=True)
        # keep acc with correct assigned tax
        df_in_fdr_species_tax_acc = get_taxon_specific_df(df_in_fdr_species_tax_acc, 'species', taxids_species, acc_to_taxid_dict)
        df_in_fdr_family_tax_acc = get_taxon_specific_df(df_in_fdr_family_tax_acc, 'family', taxid_family, acc_to_taxid_dict) 

        #print(df_in_fdr_family_186817_acc.head(10))
        # all accs
        try:
            accs_spec = sorted([item for sublist in df_in_fdr_species_tax_acc.Protein for item in sublist])
            accs_fam = sorted([item for sublist in df_in_fdr_family_tax_acc.Protein for item in sublist])
        except:
            print('error: empty df? ', df_in_fdr_species_tax_acc)

        print(f"identified spectra species for taxon {taxids_species}: {len(set(df_in_fdr_species_tax_acc.Title))}\nidentified spectra family for taxon {taxid_family}: {len(set(df_in_fdr_family_tax_acc.Title))}")
        print(f"identified accs species for taxon {taxids_species}: {len(set(accs_spec))}\nidentified accs family for taxon {taxid_family}: {len(set(accs_fam))}")
    
        #count    
        count_dict_spec = count_accs(accs_spec)
        print(f"Anzahl Protein-Identifikationen auf Species Level für taxon {taxids_species}\n\
        mit 1 identifikation:{count_dict_spec[1]}\n\
        mit 2 identifikationen: {count_dict_spec[2]},\n\
        mit 3 identifikationen: {count_dict_spec[3]}, \n\
        mit mehr als 3 Identifikationen: {count_dict_spec['>3']}")     

        count_dict_fam = count_accs(accs_fam)
        print(f"Anzahl Protein-Identifikationen auf Family Level für taxon {taxid_family}\n\
        mit 1 identifikation:{count_dict_fam[1]}\n\
        mit 2 identifikationen: {count_dict_fam[2]},\n\
        mit 3 identifikationen: {count_dict_fam[3]}, \n\
        mit mehr als 3 Identifikationen: {count_dict_fam['>3']}") 

        print()
        
        print("Eigenschaften der neu identifizierten Spektra:")
        df_family_tax_fam_not_in_species_df=df_in_fdr_family_tax_acc[~df_in_fdr_family_tax_acc.Title.isin(df_in_fdr_species_tax_acc.Title)]
        
        print(f"all new identified spectra family: {len(set(df_family_tax_fam_not_in_species_df.Title))}")
        df_only_one_taxa_tax_fam = df_family_tax_fam_not_in_species_df[get_taxa_rows(df_family_tax_fam_not_in_species_df.taxID_family, taxid_family)]
        df_only_one_taxa_tax_fam = df_family_tax_fam_not_in_species_df[get_taxa_rows(df_family_tax_fam_not_in_species_df.taxID_family, taxid_family)]
        taxa_count_dict_tax_fam = count_taxa(df_family_tax_fam_not_in_species_df.taxID_family)
        if len(df_only_one_taxa_tax_fam) != 0:
            print(f"identified exclusive for taxon {taxid_family}: {len(set(df_only_one_taxa_tax_fam.Title))}\nnot exclusive identified for taxon {taxid_family}, taxid to PSM number dict: \n{taxa_count_dict_tax_fam}")
        if len(df_family_tax_fam_not_in_species_df) != 0:
            df_family_tax_fam_in_species_df = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
            if len(df_family_tax_fam_in_species_df) != 0:
                print(f"Anzahl Spekten bereits vorher für andere Spezien identifiziert: {len(set(df_family_tax_fam_in_species_df.Title))}")

In [ ]:
import os
import urllib


#uni_info = os.system(f"wget -q -O- {url}")
def get_uniprot_information_from_api(uni_id):
    url = f"https://www.uniprot.org/uniprot/{uni_id}.txt"
    response = urllib.request.urlopen(url)
    data = response.read()      # a `bytes` object
    uni_info = data.decode('utf-8')
    return uni_info

def get_species_information_from_info(txt):
    info_dict = {}
    for line in txt.split('\n'):
        elem = line.split()
        try:
            if elem[0] == 'OX':
                info_dict["OX"]=elem[1]
                return info_dict
            elif elem[0] == "OS":
                info_dict["OS"]=elem[1:]
        except IndexError:
            print('acc not valid')
    
def get_lineage(txt):
    # OC   Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi; Mammalia;
    # OC   Eutheria; Euarchontoglires; Glires; Lagomorpha; Leporidae; Oryctolagus.
    lineage = []
    for line in txt.split('\n'):
        elem = line.split()
        elem = [lin.replace(";", "") for lin in elem]
        try:
            if elem[0] == 'OC':
                lineage+=(elem[1:])
            if elem[0] == "OX":
                return lineage
        except IndexError:
            return('acc not valid')

In [ ]:
fdr=0.05
df_in_fdr_uniprot_species = get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['species'], fdr, columns=['taxID_species']) 
df_in_fdr_uniprot_family =  get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['family'], fdr, columns=['taxID_family'])


In [ ]:
all_accs={item for sublist in df_in_fdr_uniprot_species.Protein for item in sublist}
all_accs=all_accs.union({item for sublist in df_in_fdr_uniprot_family.Protein for item in sublist})
final_accs = set()
for acc in all_accs:
    try:
        final_accs.add(acc.split('|')[1]) 
    except:
        final_accs.add(acc)
print(len(all_accs))
acc2tax_reader=ReadAccTaxon("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/", "uniprot")
acc_to_taxid_dict = acc2tax_reader.read_acc2tax(final_accs)
acc_to_taxid_dict = {key: int(taxid) for key, taxid in acc_to_taxid_dict.items()}
print(len(acc_to_taxid_dict))

In [ ]:
df_in_fdr_uniprot_species.Protein = df_in_fdr_uniprot_species.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})
df_in_fdr_uniprot_family.Protein = df_in_fdr_uniprot_family.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})


In [ ]:
df_in_fdr_uniprot_species = sort_taxid_and_acc_in_df(df_in_fdr_uniprot_species, 'species', acc_to_taxid_dict)
df_in_fdr_uniprot_family = sort_taxid_and_acc_in_df(df_in_fdr_uniprot_family, 'family', acc_to_taxid_dict)

df_in_fdr_uniprot_family

In [ ]:
#example_df.Protein = example_df.Protein.apply(lambda acc_set: {acc.split('|')[1] for acc in acc_set})
example_df = sort_taxid_and_acc_in_df(example_df, 'species', acc_to_taxid_dict)
example_df

In [ ]:
wfg = remove_all_once_appearing_accs(example_df.copy(deep=True), 'species')
wfg

In [ ]:
df_in_fdr_uniprot_species_without_one_hits = remove_all_once_appearing_accs(df_in_fdr_uniprot_species.copy(deep=True), 'species')
df_in_fdr_uniprot_family_without_one_hits = remove_all_once_appearing_accs(df_in_fdr_uniprot_family.copy(deep=True), 'family')

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
# bacillus subtilis spec:1423 fam: 186817 ohne nur einmal auftretene Accs
describe_difference_between_species_and_family_identification("bacillus subtilis", [1423], [186817], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)


In [ ]:
describe_difference_between_species_and_family_identification("staphylococcus aureus", [1280], [90964], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)
#staphylococcus aureus

In [ ]:
describe_difference_between_species_and_family_identification("Paracoccus denitrificans", [266], [31989], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)
# Paracoccus denitrificans

In [ ]:
describe_difference_between_species_and_family_identification("chromobacterium violaceum", [536], [1499392], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)
#chromobacterium violaceum

In [ ]:
describe_difference_between_species_and_family_identification("altermonas macleodii", [28108], [72275], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)
#altermonas macleodii

In [ ]:
describe_difference_between_species_and_family_identification("paraburkholderia", [36873, 119219], [119060], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)
#paraburkholderia

In [ ]:
# loading 9MM species and family:
fdr=0.05
tanca_species_df = get_df_in_fdr_without_decoy("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv/9MM_FASP_uniprot_Tanca_species_nr.t.xml_new_reduced.tsv", fdr, columns=['taxID_species'])
tanca_family_df = get_df_in_fdr_without_decoy("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv/9MM_FASP_uniprot_Tanca_family_nr.t.xml_new_reduced.tsv", fdr, columns=['taxID_family'])
tanca_species_df.Protein = tanca_species_df.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})
tanca_family_df.Protein = tanca_family_df.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})

In [ ]:
tanca_species_df_without_one_hits = remove_all_once_appearing_accs(tanca_species_df.copy(deep=True))
tanca_family_df_without_one_hits = remove_all_once_appearing_accs(tanca_family_df.copy(deep=True))

In [ ]:
describe_difference_between_species_and_family_identification("rhodotorula glutinis", 5535, 1799696, tanca_species_df, tanca_family_df, tanca_species_df_without_one_hits, tanca_family_df_without_one_hits)
# rhodotorula glutinis
# identified spectra species: 15 identified spectra family: 495

In [ ]:
# viruses
virus_family_IDs = [10744, 10699, 10662, 11989]
virus_species_IDs = [10754, 101570, 1283336, 12022, 1977402]
# da fehler drin: df_in_fdr_uniprot_family

describe_difference_between_species_and_family_identification("viruses", virus_species_IDs, virus_family_IDs, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)


In [ ]:
describe_difference_between_species_and_family_identification("Pseudomonas", [294, 1149133, 1294143], [135621], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)

In [ ]:
Kleiner_taxIDs = [536, 882, 44577, 228410, 323848, 46170, 93061, 224308, 99287, 511145, 176299, 216596, 1041145,
                      262724, 266264, 266265, 1004788, 1114970, 1149133, 1294143, 1407502, 3055, 1302247, 926571,
                      # virus
                      10754, 101570, 1283336, 12022, 1977402]
Tanca_taxIDs = [747, 5535, 655183, 1579, 1255, 4932, 1465, 1351, 562]

In [ ]:
describe_difference_between_species_and_family_identification("Rhizobium", [44577, 1231, 915],  [206379], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)


In [ ]:

describe_difference_between_species_and_family_identification("Nitrosomonadaceae", [384, 1176649],  [82115], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)


In [ ]:
describe_difference_between_species_and_family_identification("Enterobacteriaceae", [28901, 562],  [543], df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)


In [ ]:
Kleiner_2 = [("Desulfovibrio vulgaris", [881]), ("Nitrosomonas ureae", [44577]),("Thermus thermophilus", [274]),\
("Stenotrophomonas maltophilia", [40324]), ("Chlamydomonas reinhardtii", [3055]),  ("Nitrososphaera viennensis", [1034015])]

In [ ]:
for name, taxon_species in Kleiner_2:
    taxon_family = [taxon_graph.find_level_up(taxon_species[0], "family")]
    describe_difference_between_species_and_family_identification(name, taxon_species, taxon_family, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, acc_to_taxid_dict)


In [ ]:
Kleiner = [("Desulfovibrio vulgaris", [881]), ("Nitrosomonas ureae", [44577]),("Thermus thermophilus", [274]),\
("Stenotrophomonas maltophilia", [40324]), ("Chlamydomonas reinhardtii", [3055]),  ("Nitrososphaera viennensis", [1034015])]

Kleiner_3 = [("Enterobacteriaceae", [28901, 562],  [543]), ("Nitrosomonadaceae", [384, 1176649],  [82115]),\
            ("Rhizobium", [44577, 1231, 915],  [206379]), ("Pseudomonas", [294, 1149133, 1294143], [135621]),\
             ("viruses", [10754, 101570, 1283336, 12022, 1977402], [10744, 10699, 10662, 11989]),\
             ("paraburkholderia", [36873, 119219], [119060]), ("altermonas macleodii", [28108], [72275]), \
             ("chromobacterium violaceum", [536], [1499392]), ("Paracoccus denitrificans", [266], [31989]), \
             ("staphylococcus aureus", [1280], [90964]), ("bacillus subtilis", [1423], [186817])]
             
Tanca = [("rhodotorula glutinis", [5535])]

In [ ]:
all_taxon_identified_spectra_spec, exclusive_spectra_spec = taxon_exclusive_spectra(df_in_fdr_uniprot_species, "taxID_species", 1280)
all_taxon_identified_spectra_fam, exclusive_spectra_fam = taxon_exclusive_spectra(df_in_fdr_uniprot_family, "taxID_family", 90964)
new_identified_spectra, spectra_in_species_df = spectra_identified_in_df(df_in_fdr_uniprot_species, df_in_fdr_uniprot_family, 1280, 90964)
print(f"all_taxon_identified_spectraspec: {len(all_taxon_identified_spectra_spec)}, exclusive_spectra_spec: {len(exclusive_spectra_spec)}, all_taxon_identified_spectra_fam: {len(all_taxon_identified_spectra_fam)}, exclusive_spectra_fam: {len(exclusive_spectra_fam)}")
print(f"new_identified_spectra: {len(new_identified_spectra)}, spectra_in_species_df: {len(spectra_in_species_df)}, spectra unknown in species df: {len(new_identified_spectra) - len(spectra_in_species_df)}")      

In [ ]:
accs = {'A0A2N5HN64', 'A0A3Q9R1C8', 'A0A0M4BU31', 'A0A2B9AMC6', 'E0U2U6', 'A0A4R6U1N0', 'A0A160F5P8', 'A0A1Q9B947', 'A0A1C3SJW4', 'A0A150MTL3', 'A0A0D6ZE08', 'A0A4R3KF72', 'A0A0D1KBI9', 'A0A285D020', 'A0A268ED93', 'A0A448FF24', 'S5ZSF5', 'A0A080UWE9', 'M8D4P2', 'A0A1V9BDP9', 'A0A384IMU6', 'A0A250E382', 'A0A226QK25', 'A0A4S3PWA9', 'A0A1H9X6F1', 'G9QJP1', 'A0A323TDM3', 'A0A2C0ZKD5', 'A0A133KBV2', 'A0A3M7TPM5', 'A0A2S4F874', 'A0A420VHU6', 'A0A250E2M5', 'A0A0N1F2B4', 'A0A142D3Q8', 'K2MJP6', 'A0A0J6EM84', 'A0A1Q5SMI9', 'A0A495A814', 'A0A2A8MIB6', 'A0A263BR38', 'A0A3A9KUZ0', 'A0A2T0DA05', 'A0A0Q0YVS7', 'A0A0N0Z530', 'B7GL24', 'A0A1Y0YSR3', 'A0A368WK05', 'A6CTF9', 'A0A3D0E6N8', 'A0A0M4FRW3', 'A0A1S2M8H0', 'A0A0M0KGS9', 'A0A0A6Y2H5', 'A0A150F767', 'A0A150MXB0', 'A0A0N8GGV1', 'A0A1B7KQJ8', 'A0A1S1G6H2', 'A0A2A5IRY5', 'A0A160FAQ0', 'A0A398B4J6', 'A0A2S7MZY0', 'A0A0B0HIJ8', 'A0A098F5L2', 'W6LP63', 'A0A063YPU2', 'A0A1D7QSI0', 'A0A366Y057', 'A0A1H0WBG7', 'I3DUA7', 'A0A0F6BIK8', 'A0A4R1QEH3', 'L8A2U7', 'A0A3C2CVM7', 'A0A1S0Z749', 'A0A2V2HJQ7', 'A0A3A1R7K0', 'A0A0M2EKP8', 'A0A0D0RXY4', 'I3E288', 'A0A150MC37', 'A0A410XGS3', 'A0A1H3SA94', 'A0A0U1NTU4', 'A0A146CMK5', 'A0A2C3K8F3', 'A0A072NSP6', 'A0A1A0FT78', 'A0A087LAQ9', 'A0A4V6NJG8', 'A0A4R3MS70', 'A0A4R2BI63', 'A0A2P1FA58', 'A0A023CIE5', 'S7SRY7', 'A0A1I6CBK3', 'A0A1Y0CRS8', 'A0A410WCD2', 'A0A0E0TFI0', 'C5D7M1', 'A0A094JVX0', 'A0A429Y841', 'A0A2A9BTQ8', 'A0A1V9B146', 'A0A135L5T6', 'A0A163ZNQ2', 'A0A363E086', 'A0A1D2YVF0', 'A0A0M9GSS3', 'A0A0J5TE70', 'A8FHJ0', 'A0A1U6JN39', 'W4RTD1', 'W7RQP0', 'A0A2B1RS58', 'G2TP79', 'A0A150KIK9', 'A0A412GVE9', 'A0A150JNB9', 'A0A0H4NVX0', 'A0A502BBZ6', 'M5NZB7', 'A0A4Q0VW86', 'A0A178U422', 'A0A1V3FGV4', 'A0A366ET50', 'A0A150N7M1', 'A0A2N5FFL2', 'A0A370GF92', 'A0A4U1LLF0', 'A0A165YRK0', 'A0A1R1S081', 'A0A084GXK8', 'T5HNG2', 'A0A2A9RNU2', 'A0A3C1I437', 'A0A2T5SE07', 'A0A380ERN0', 'A0A4S3L7F6', 'A0A0Q3W998', 'A0A0A8X7G4', 'A0A4R5VTF7', 'A0A2B8U3F4', 'A0A2A7WWI6', 'A0A1V9CAB8', 'A0A1S2RF42', 'K6DHF4', 'A0A177ZM05', 'A0A223KUT2', 'A0A3M0EWV8', 'A0A2X4WFM0', 'A0A199WEN2', 'A0A1N7J2B7', 'A0A2S9WP15', 'A0A0D8BUC5', 'A0A0K9HJ26', 'A0A223FF36', 'A0A4Y9K4B6', 'A0A1J6VUU7', 'A0A4Y7WKF3', 'A0A429X5C2', 'U2Y675', 'A0A1Q8UQP5', 'A0A1S2MHD0', 'A0A0A2UUE4', 'A0A3E0JWZ6', 'A0A2A7IHD8', 'A0A0J5GWW8', 'A0A1I0SWX4', 'A0A327YGJ6', 'A0A178TKG8', 'A0A0M2SXX4', 'A0A3R9NHP9', 'A0A223DVS5', 'A0A1G8V933', 'A0A023DH73', 'A0A0V8HBW8', 'A0A0K9G9Q6', 'A0A3L7K2Z0', 'A0A1J5WVP0', 'A0A147K5U5', 'A0A1I0TYV6'}



In [ ]:

            
for acc in accs:       
    uni_info = get_uniprot_information_from_api(acc)
    lineage = get_lineage(uni_info)
    if lineage[0] == "Viruses":
        print(acc)
        print(lineage)
    #W6LP63

In [ ]:
uni_info = get_uniprot_information_from_api("W6LP63")
lineage = get_lineage(uni_info)
print(lineage)

In [ ]:
virus_family_IDs = [10744, 10699, 10662, 11989]
virus_species_IDs = [10754, 101570, 1283336, 12022, 1977402]
df_virus_fam = df_in_fdr_uniprot_family_without_one_hits[get_taxa_rows(df_in_fdr_uniprot_family_without_one_hits.taxID_family, virus_family_IDs)]
df_exclusive_virus_fam = df_in_fdr_uniprot_family_without_one_hits[get_exclusive_rows(df_in_fdr_uniprot_family_without_one_hits.taxID_family, virus_family_IDs)]
df_without_one_hit_peptides = remove_all_accs_with_only_one_peptide(df_virus_fam)
print(len(df_without_one_hit_peptides))
all_accs = df_without_one_hit_peptides.Protein
for acc_set in all_accs:
    print(len(acc_set))
    for acc in acc_set:
        uni_info = get_uniprot_information_from_api(acc)
        lineage = get_lineage(uni_info)
        if lineage[0] == "Viruses":
            print(acc)
            print(lineage)
    

In [ ]:
virus_family_IDs = [10744, 10699, 10662, 11989]
virus_species_IDs = [10754, 101570, 1283336, 12022, 1977402]
df_virus_fam = df_in_fdr_uniprot_family_without_one_hits[get_taxa_rows(df_in_fdr_uniprot_family_without_one_hits.taxID_family, virus_family_IDs)]
df_exclusive_virus_fam = df_in_fdr_uniprot_family_without_one_hits[get_exclusive_rows(df_in_fdr_uniprot_family_without_one_hits.taxID_family, virus_family_IDs)]
df_without_one_hit_peptides = remove_all_accs_with_only_one_peptide(df_virus_fam)
df_without_one_hit_peptides

In [ ]:
for name, taxon_species in Kleiner:
    taxon_family = taxon_graph.find_level_up(taxon_species, "family")
    all_taxon_identified_spectra_spec, exclusive_spectra_spec = taxon_exclusive_spectra(df_in_fdr_uniprot_species_without_one_hits, "taxID_species", taxon_species)
    all_taxon_identified_spectra_fam, exclusive_spectra_fam = taxon_exclusive_spectra(df_in_fdr_uniprot_family_without_one_hits, "taxID_family", taxon_family)
    new_identified_spectra, spectra_in_species_df = spectra_identified_in_df(df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, taxon_species, taxon_family)
    print(name)
    print(f"all_taxon_identified_spectra species: {len(all_taxon_identified_spectra_spec)}\nall_taxon_identified_spectra family: {len(all_taxon_identified_spectra_fam)}\nexclusive_spectra species: {len(exclusive_spectra_spec)} \nexclusive_spectra family: {len(exclusive_spectra_fam)}")
    print(f"intersection species and family exclusive spectra: {len(exclusive_spectra_spec.intersection(exclusive_spectra_fam))}")    
    print(f"new_identified_spectra: {len(new_identified_spectra)} \nnew_identified_spectra bereits in species_df: {len(spectra_in_species_df)} \nspectra unknown in species df: {len(new_identified_spectra) - len(spectra_in_species_df)}")      
    print()

In [ ]:
for name, taxon_species, taxon_family in Kleiner_3:
    all_taxon_identified_spectra_spec, exclusive_spectra_spec = taxon_exclusive_spectra(df_in_fdr_uniprot_species_without_one_hits, "taxID_species", taxon_species)
    all_taxon_identified_spectra_fam, exclusive_spectra_fam = taxon_exclusive_spectra(df_in_fdr_uniprot_family_without_one_hits, "taxID_family", taxon_family)
    new_identified_spectra, spectra_in_species_df = spectra_identified_in_df(df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, taxon_species, taxon_family)
    print(name)
    print(f"all_taxon_identified_spectra species: {len(all_taxon_identified_spectra_spec)}\nall_taxon_identified_spectra family: {len(all_taxon_identified_spectra_fam)}\nexclusive_spectra species: {len(exclusive_spectra_spec)} \nexclusive_spectra family: {len(exclusive_spectra_fam)}")
    print(f"intersection species and family exclusive spectra: {len(exclusive_spectra_spec.intersection(exclusive_spectra_fam))}")    
    print(f"new_identified_spectra: {len(new_identified_spectra)} \nnew_identified_spectra bereits in species_df: {len(spectra_in_species_df)} \nspectra unknown in species df: {len(new_identified_spectra) - len(spectra_in_species_df)}")      
    print()

In [ ]:
Kleiner_2 = [("Desulfovibrio vulgaris", [881]), ("Nitrosomonas ureae", [44577]),("Thermus thermophilus", [274]),\
("Stenotrophomonas maltophilia", [40324]), ("Chlamydomonas reinhardtii", [3055]),  ("Nitrososphaera viennensis", [1034015])]

Kleiner_3 = [("Enterobacteriaceae", [28901, 562],  [543]), ("Nitrosomonadaceae", [384, 1176649],  [82115]),\
            ("Rhizobium", [44577, 1231, 915],  [206379]), ("Pseudomonas", [294, 1149133, 1294143], [135621]),\
             ("viruses", [10754, 101570, 1283336, 12022, 1977402], [10744, 10699, 10662, 11989]),\
             ("paraburkholderia", [36873, 119219], [119060]), ("altermonas macleodii", [28108], [72275]), \
             ("chromobacterium violaceum", [536], [1499392]), ("Paracoccus denitrificans", [266], [31989]), \
             ("staphylococcus aureus", [1280], [90964]), ("bacillus subtilis", [1423], [186817])]

Tanca = [("rhodotorula glutinis", 5535)]

def write_results(out, all_identified_spectra_spe, all_identified_spectra_no_one_hits_spe, exclusive_spectra_spe, \
                  exclusive_spectra_no_one_hits_spe, all_identified_spectra_fam, all_identified_spectra_no_one_hits_fam, \
                  exclusive_spectra_fam, exclusive_spectra_no_one_hits_fam, new_identified_spectra, spectra_in_species_df):
        out.write(f"all identified_spectra for taxa level species\t{len(all_identified_spectra_spe)}\n")
        out.write(f"all identified_spectra for taxa level family\t{len(all_identified_spectra_fam)}\n")
        out.write(f"exclusive spectra for taxa level species\t{len(exclusive_spectra_spe)}\n")
        out.write(f"exclusive spectra for taxa level family\t{len(exclusive_spectra_fam)}\n")
        out.write(f"intersection species and family exclusive spectra\t{len(exclusive_spectra_spec.intersection(exclusive_spectra_fam))}")    
        
        out.write(f"all identified_spectra for taxa level species no one hits\t{len(all_identified_spectra_no_one_hits_spe)}\n")
        out.write(f"all identified_spectra for taxa level family no one hits\t{len(all_identified_spectra_no_one_hits_fam)}\n")
        out.write(f"exclusive spectra for taxa level species no one hits\t{len(exclusive_spectra_no_one_hits_spe)}\n")
        out.write(f"exclusive spectra for taxa level family no one hits\t{len(exclusive_spectra_no_one_hits_fam)}\n")
        out.write(f"intersection species and family exclusive spectra no one hits\t{len(exclusive_spectra_no_one_hits_spe.intersection(exclusive_spectra_no_one_hits_fam))}")    
        
        out.write(f"new_identified_spectra no one hits\t{len(new_identified_spectra)}\n")
        out.write(f"new_identified_spectra bereits in species_df no one hit: {len(spectra_in_species_df)}\n")
        out.write(f"spectra unknown in species df no one hits\t{len(new_identified_spectra) - len(spectra_in_species_df)}")      
        out.write("\n")

with open("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_analysis/results_per_taxa.tsv", "w") as out:
    out.write("Analysis of identifed spectra")

with open("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_analysis/results_per_taxa.tsv", "a") as out:   
    for name, taxon_species in Kleiner_2:
        taxon_family = taxon_graph.find_level_up(taxon_species[0], "family")
        out.write(f"{name}\t{taxon_species[0]}\t{taxon_family}\n")
        all_identified_spectra_spe, all_identified_spectra_no_one_hits_spe, exclusive_spectra_spe, \
        exclusive_spectra_no_one_hits_spe = taxon_exclusive_spectra(df_in_fdr_uniprot_species, "taxID_species", taxon_species)
        all_identified_spectra_fam, all_identified_spectra_no_one_hits_fam, exclusive_spectra_fam, exclusive_spectra_no_one_hits_fam = \
        taxon_exclusive_spectra(df_in_fdr_uniprot_family, "taxID_family", taxon_family)
        new_identified_spectra, spectra_in_species_df = spectra_identified_in_df(df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, taxon_species, taxon_family)    
        write_results(out, all_identified_spectra_spe, all_identified_spectra_no_one_hits_spe, exclusive_spectra_spe, \
                  exclusive_spectra_no_one_hits_spe, all_identified_spectra_fam, all_identified_spectra_no_one_hits_fam, \
                  exclusive_spectra_fam, exclusive_spectra_no_one_hits_fam, new_identified_spectra, spectra_in_species_df)
    
    for name, taxon_species, taxon_family in Kleiner_3:
        out.write(f"{name}\t{taxon_species[0]}\t{taxon_family}\n")
        all_identified_spectra_spe, all_identified_spectra_no_one_hits_spe, exclusive_spectra_spe, exclusive_spectra_no_one_hits_spe = \
        taxon_exclusive_spectra(df_in_fdr_uniprot_species, "taxID_species", taxon_species)
        all_identified_spectra_fam, all_identified_spectra_no_one_hits_fam, exclusive_spectra_fam, exclusive_spectra_no_one_hits_fam = \
        taxon_exclusive_spectra(df_in_fdr_uniprot_family, "taxID_family", taxon_family)
        new_identified_spectra, spectra_in_species_df = spectra_identified_in_df(df_in_fdr_uniprot_species_without_one_hits, df_in_fdr_uniprot_family_without_one_hits, taxon_species, taxon_family)    
        write_results(out, all_identified_spectra_spe, all_identified_spectra_no_one_hits_spe, exclusive_spectra_spe, \
                  exclusive_spectra_no_one_hits_spe, all_identified_spectra_fam, all_identified_spectra_no_one_hits_fam, \
                  exclusive_spectra_fam, exclusive_spectra_no_one_hits_fam, new_identified_spectra, spectra_in_species_df)
    
    